# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


## Turaev cobraket

In [ ]:
def cobracket(myw, verbose: false, narrow: true)
  
  monomials = 
  .map do |mono|

    mono.merge({coeff: mono[:linking], words: words})
  end

  if verbose
    result = monomials
  else
    result = []
    while monomials.size > 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      #---
      monomials.each do |mono|
        if mono[:words][0].conjugate?(pop[:words][0]) and mono[:words][1].conjugate?(pop[:words][1])
          pop[:coeff] += mono[:coeff]
          pop[:divs] << mono[:divs]
          mono[:coeff] = nil
        elsif mono[:words][0].conjugate?(pop[:words][1]) and mono[:words][1].conjugate?(pop[:words][0])
          pop[:coeff] += mono[:coeff]*(-1)
          pop[:divs] << mono[:divs]
          mono[:coeff] = nil
        end
      end
      #---
      if pop[:coeff] != 0 or verbose
        result << pop
      end
      #---
      monomials.delete_if{|mono| mono[:coeff].nil?}
    end 
  end
  return (result.empty?) ? {coeff: 0, words: []} : result
end; nil

In [ ]:
def ep(letter1, letter2, bar=false)
  result = if letter1.show == letter2.show
    (bar) ? -1 : 1        
  else
    (Rseq.index(letter1) < Rseq.index(letter2)) ? 1 : -1
  end
  return result
end

def lk(myw, idxs)
  facs = myw.flatten.factors
  #---
  divs =  idxs.map{|k| [facs[(k+1)/2-1], facs[((k+1)/2)%myw.size]]}
  lts = divs.map{|pair| [pair[1], pair[0].inverse]}
  arr = [
    [lts[0][0], lts[1][0], false], 
    [lts[0][0], lts[1][1], true], 
    [lts[0][1], lts[1][0], false], 
    [lts[0][1], lts[1][1], true]
    ]
  vals = arr.map{|a| ep(a[0], a[1], a[2])}
  #return (0..arr.size-1).map{|i| "#{arr[i][0].show}.#{arr[i][1].show} = #{vals[i]}"}
  return (vals[0] - vals[1] - vals[2] + vals[3])/2
end

def show_linkings(myw)
vb = true #false #
  
facs = myw.flatten.factors
#p facs.map(&:show)
printf("=====\n")

cobra = cobracket(myw, verbose: vb)
linking_sum = 0

cobra.each do |h|
  h.each do |k,v|
    case k 
    when :odds
      str = facs.map(&:show).join
      idx = v.map{|n| (n+1)/2}
      str = [str[0..idx[0]-1], str[idx[0]..idx[1]-1], str[idx[1]..-1]].join('|')
      puts "#{k}: #{v},  #{str}"
      p lk(myw, v)
    when :linking
      p "#{k}: #{v}"
    when :factors
#      p h[:factors].map(&:show).join(" , ")
    when :words
#      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
#      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  linking_sum += h[:linking]
  printf("----------\n")
end
return "linking_sum = #{linking_sum}"
end; nil

# Execution

In [ ]:
a1 = Word.new(a[1])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = {
  s0: [
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse)
  ],
  s1: [
    a1^4,
    c,
    comms[1]
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse)
  ],
  s4: [
    a1*(a2^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2])
  ]
  }

arr = samples #.slice(:s2, :s3) #:s0, :s1)
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each do |s|
    ws = s.show
    cob = cobracket(s).map do |h|
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      coeff + "#{h[:words].join(" \u{2227} ")}"
    end
    #---
    puts "#{ws}  |--\u{03B4}-->  #{cob.join(' + ')}"
  end
end;nil

In [ ]:
myw = samples[:s6][-1]
wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = true
printf("=====\n")
cobracket(myw, verbose: vb).each do |h|
  h.each do |k,v|
    case k 
    when :odds, :linking
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  printf("----------\n")
end; nil

## Experiments

In [ ]:
ws = [a1*(comms[2].inverse), a1*(c.inverse)*(comms[2].inverse)*c]
wcs = ws.map{|w| Wcode.new(w)}
wcs.each{|wc| p "#{wc.word.show} --> #{wc.show(3)}" }

cobras = ws.map{|w| cobracket(w, verbose: true, narrow: false)} # false)} #
printf "\n\n"
#---

(1..ws[0].size).map{|n| 2*n-1}.combination(2).each do |op|
  op2 = op.map{|o| (o == 1) ? o : o+6 }
  c0 = cobras[0].select{|h| h[:odds]==op}
  c1 = cobras[1].select{|h| h[:odds]==op2}

[c0, c1].each do |c|
  c.each do |h|
    h.each do |k,v|
      case k 
      when :odds, :linking
        p "#{k}: #{v}"
      when :factors
        p h[:factors].map{|f| f.show(3)}.join(" , ")
      when :words
        coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
        p coeff + "#{h[:words].join(" \u{2227} ")}"
      else
      end
    end
    printf("----------\n")
  end;
end
  printf("=================\n")
end; nil

In [ ]:
myw = a1*(comms[2].inverse)
wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = true
printf("=====\n")
cobracket(myw, verbose: vb).each do |h|
  h.each do |k,v|
    case k 
    when :odds, :linking
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  printf("----------\n")
end; nil

In [ ]:
p myw.show
d1 = (a1.size)+1
d2 = cif + c.size
d3 = myw.size - (c.size-1)
d4 = myw.size + 1

r1 = (d1..d2-1).map{|n| 2*n-1}.to_a
r2 = (d3..d4-1).map{|n| 2*n-1}.to_a
p r1, r2
p r1.include?(9)
nil

In [ ]:
myw = a1*(c.inverse)*(comms[2].inverse)*c
wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = true #false #
printf("=====\n")

cobra = cobracket(myw, verbose: vb)

d1 = (a1.size)+1
d2 = cif + c.size
d3 = myw.size - (c.size-1)
d4 = myw.size + 1
r1 = (d1..d2-1).map{|n| 2*n-1}.to_a
r2 = (d3..d4-1).map{|n| 2*n-1}.to_a
rng = r1+r2
cobra.select! do |h|
  rng.include?(h[:odds][0]) or rng.include?(h[:odds][1])
end

cobra.each do |h|
  cnt = 0
  str = ''
#  str += myw.show.each_char do |c|
#    unless c == '.'
#      cnt += 1
#      c = "***" #if (h[:odds].include?(2*cnt-1))
#    end
#    '|||'
#  end
  str = myw.show.delete('.')
  idx = h[:odds].map{|n| (n+1)/2}
  str = [str[0..idx[0]-1], str[idx[0]..idx[1]-1], str[idx[1]..-1]].join('|')
  puts "#{h[:odds]}, #{str}"
  h.each do |k,v|
    case k 
    when :linking #, :odds 
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  printf("----------\n")
end; nil

### TODO

* [ ] Wcode#devide して得られる2つの Wcode instance の show(2) (および show(3)) がバグってる。なぜか each_slice(4) が2つずつくくる。
* [ ] 係数がプラマイ逆かな。
* [x] simple subword たちによる計算を実装する。<-- ダメだった。各 simple subword が内部では確実に交差しない保証がないとダメだから。